# A. Extract Features

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from submission.utils.utils import extract_features_from_image, perform_pca, train_svm_model

# custom preprocessing
#from submission.utils.image_preprocessing import preprocess_image

## A.1. Extract Features for Multiclass

In [9]:
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import cv2
import numpy as np
from pathlib import Path


BASE_PATH = str(Path.cwd().resolve())
#print(BASE_PATH)

#BASE_PATH = "/datax/Maack/ism_2024_2025/sample_solution/phase_1a"
PATH_TO_GT = os.path.join(BASE_PATH, "gt_for_classification_multiclass_from_filenames_0_index.csv")
PATH_TO_IMAGES = os.path.join(BASE_PATH, "images")

df = pd.read_csv(PATH_TO_GT)

images = df["file_name"].tolist()

features = []
labels = []

for i in range(len(df)):
    
    image_name = df.iloc[i]["file_name"]
    label = df.iloc[i]["category_id"]

    path_to_image = os.path.join(PATH_TO_IMAGES, image_name)
    image = cv2.imread(path_to_image)

    ### ADDED AN IMAGE PREPROCESSING STEP ###
    #image = preprocess_image(image, IMAGE_PATH=path_to_image)
    #########################################


    image_features = extract_features_from_image(image)
    
    features.append(image_features)
    labels.append(label)
    
features_multiclass = np.array(features)
labels_multiclass = np.array(labels)

print("Features shape:", features_multiclass.shape)
print("Labels shape:", labels_multiclass.shape)
print()
print(labels_multiclass)

Features shape: (2845, 9755)
Labels shape: (2845,)

[1 1 1 ... 1 2 1]


## B.2. Use Prinicpal Component Anaylsis to reduce dimensionality

In [10]:
k = 100


features_multiclass_reduced = perform_pca(features_multiclass, k)

# C. Train Classification Model for Multiclass

In [11]:
# multiclass_model = train_svm_model(features_multiclass, labels_multiclass)
multiclass_model = train_svm_model(features_multiclass_reduced, labels_multiclass)

Train Accuracy: 0.9991
Test Accuracy:  1.0000
Test F1-score:  1.0000


In [12]:
# save the weights of multiclass_model
import pickle


#SAVE_PATH = "/datax/Maack/ism_2024_2025/sample_solution/phase_1a/submission" # change this to your own path
SAVE_PATH = os.path.join(BASE_PATH, "submission")

with open(os.path.join(SAVE_PATH, "multiclass_model.pkl"), "wb") as f:
    pickle.dump(multiclass_model, f)